In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F


In [3]:
transform = transforms.Compose([transforms.Resize((224,224)) , transforms.ToTensor()])
transform

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
)

In [4]:
trainset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/flowers' , transform =transform)

In [50]:
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=16,
                                          shuffle=True,
                                          num_workers=2)

In [34]:
testset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/flowers', transform = transform)

In [51]:
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=16,
                                         shuffle=False,
                                         num_workers=2)

In [8]:
labels = ('daisy' , 'dandelion' , 'rose' , 'sunflower' , 'tulip')

In [9]:
images_batch, labels_batch = iter(trainloader).next()

In [10]:
images_batch.shape

torch.Size([16, 3, 224, 224])

In [ ]:
#img = torchvision.utils.make_grid(images_batch)

In [11]:
import torch.nn as nn

In [36]:
class VGG16(nn.Module):
    
    def __init__(self):
        super(VGG16, self).__init__()
        
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=64,kernel_size=(3,3)),nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=64,out_channels=64,kernel_size=(3,3)),nn.ReLU())
        nn.MaxPool2d(kernel_size=2,stride=2)
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=64,out_channels=128,kernel_size=(3,3)),nn.ReLU())
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=128,out_channels=128,kernel_size=(3,3)),nn.ReLU())
        nn.MaxPool2d(kernel_size=2,stride=2)
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=128,out_channels=256,kernel_size=(3,3)),nn.ReLU())
        self.layer6 = nn.Sequential(nn.Conv2d(in_channels=256,out_channels=256,kernel_size=(3,3)),nn.ReLU())
        self.layer7 = nn.Sequential(nn.Conv2d(in_channels=256,out_channels=256,kernel_size=(3,3)),nn.ReLU())
        nn.MaxPool2d(kernel_size=2,stride=2)
        self.layer8 = nn.Sequential(nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3)),nn.ReLU())
        self.layer9 = nn.Sequential(nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3)),nn.ReLU())
        self.layer10 =nn.Sequential(nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3)),nn.ReLU())
        nn.MaxPool2d(kernel_size=2,stride=2)
        self.layer11 = nn.Sequential(nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3)),nn.ReLU())
        self.layer12 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=512,kernel_size=(3,3)),nn.ReLU())
        self.layer13 = nn.Sequential(nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(3,3)),nn.ReLU())
        nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.fc1 = nn.Linear(25088,4096)
        self.fc2= nn.Linear(4096,4096)
        self.fc3 =nn.Linear(4096,5)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out
        

In [37]:
model = VGG16()
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model=model.to(device)

In [44]:
learning_rate =0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), 
                             lr=learning_rate)

In [48]:
total_step = len(trainloader)
num_epochs = 20
print(total_step)

68


In [52]:

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        labels=labels.to(device=device)
        images=images.to(device=device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 80 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/20], Step [80/68], Loss: 1.5889
Epoch [1/20], Step [160/68], Loss: 1.6209
Epoch [1/20], Step [240/68], Loss: 1.6093
Epoch [2/20], Step [80/68], Loss: 1.5976
Epoch [2/20], Step [160/68], Loss: 1.5322
Epoch [2/20], Step [240/68], Loss: 1.6157
Epoch [3/20], Step [80/68], Loss: 1.6228
Epoch [3/20], Step [160/68], Loss: 1.6326
Epoch [3/20], Step [240/68], Loss: 1.5627
Epoch [4/20], Step [80/68], Loss: 1.6339
Epoch [4/20], Step [160/68], Loss: 1.5827
Epoch [4/20], Step [240/68], Loss: 1.5785
Epoch [5/20], Step [80/68], Loss: 1.5936
Epoch [5/20], Step [160/68], Loss: 1.5817
Epoch [5/20], Step [240/68], Loss: 1.5951
Epoch [6/20], Step [80/68], Loss: 1.5598
Epoch [6/20], Step [160/68], Loss: 1.5442
Epoch [6/20], Step [240/68], Loss: 1.7370
Epoch [7/20], Step [80/68], Loss: 1.6245
Epoch [7/20], Step [160/68], Loss: 1.6553
Epoch [7/20], Step [240/68], Loss: 1.5834
Epoch [8/20], Step [80/68], Loss: 1.6369
Epoch [8/20], Step [160/68], Loss: 1.6379
Epoch [8/20], Step [240/68], Loss: 1.5663


In [53]:
model.eval()  
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:

        images=images.to(device=device)
        labels=labels.to(device=device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the 10000 test images: {}%'\
          .format(100 * correct / total))


Accuracy of the model on the 10000 test images: 24.368774611999072%


In [ ]:
torch.save(model.state_dict(), 'VGG16.pkl')